In [1]:
import numpy as np
import scipy.stats as st
import pandas as pd
import scipy
import warnings

import iqplot
import bebi103
import os

import bokeh.io
import bokeh.plotting
import bokeh.layouts
bokeh.io.output_notebook()

Loading BokehJS ...

## Uploading the data

1. Uploading the whole excel file to read from all the sheets later.

In [2]:
df = pd.ExcelFile(os.path.join('..', 'data', 'behavior_data', 'benCom_Motor_GI_function.xlsx'))

2. Creating a list of all sheets (tests) that we want to analyse.

In [56]:
tests = [i for i in df.sheet_names if ('Weight' not in i)]
tests

['Beam_time',
 'Beam_steps',
 'Pole',
 'Wirehang',
 'Adhesive_removal',
 'Hindlimb',
 'Fecal_output',
 'Fecal_score',
 'Water_content',
 'Gut_transit',
 'Bead_exp']

3. Parsing the Excel file into separate datasets (1 test = 1 dataset) and storing them in a dictionary with keys = names of the tests/sheets

In [4]:
data_dict = {}
for test in tests:
    temp_df = df.parse(test)   
    temp_df = temp_df.rename(columns={"Trial1": "Measurement", 
                                      "Trial15": "Measurement", 
                                      "Percent_water_content": "Measurement",
                                      "Time_min": "Measurement",
                                      "Slips_Step_Trial1": "Measurement"
                                     })
        
    temp_df = temp_df.dropna()
    data_dict[test] = temp_df

## Defining function for the MLE calculations

In [5]:
def log_like_iid_lognormal_mixed(params, n, thresh):
    """Log likelihood for i.i.d. lognormal measurements mixed with Dirac delta function.

    Parameters
    ----------
    params : array
        Parameters mu, sigma, omega.
    n : array
        Array of data points.

    Returns
    -------
    output : float
        Log-likelihood.
    """
    mu, sigma, omega = params

    if mu <= 0 or sigma <= 0 or omega <=0 or omega >= 1:
        return -np.inf

    target = 0
    
    for i in n:
        if i == thresh:
            target += scipy.special.logsumexp([np.log(1 - omega), np.log(omega) + np.log(1 - st.lognorm.cdf(i, sigma, scale=np.exp(mu)))])
        else:
            target += np.log(omega) + st.lognorm.logpdf(i, sigma, scale=np.exp(mu))                                

    return target


def log_like_iid_lognormal(params, n, thresh):
    """Log likelihood for i.i.d. lognormal measurements.

    Parameters
    ----------
    params : array
        Parameters mu, sigma.
    n : array
        Array of data points.

    Returns
    -------
    output : float
        Log-likelihood.
    """
    mu, sigma = params

    if mu <= 0 or sigma <= 0:
        return -np.inf

    target = 0
    cdf_val = np.log(1 - st.lognorm.cdf(thresh, sigma, scale=np.exp(mu)))
    
    for i in n:
        if i == thresh:
            target += cdf_val
        else:
            target += st.lognorm.logpdf(i, sigma, scale=np.exp(mu))

    return target
    

def log_like_iid_normal_mixed(params, n, thresh):
    """Log likelihood for i.i.d. normal measurements mixed with Dirac delta function.

    Parameters
    ----------
    params : array
        Parameters mu, sigma, omega.
    n : array
        Array of data points.

    Returns
    -------
    output : float
        Log-likelihood.
    """
    mu, sigma, omega = params

    if mu <= 0 or sigma <= 0 or omega <=0 or omega >= 1:
        return -np.inf

    target = 0
    cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))
    
    for i in n:
        if i == thresh:
            target += scipy.special.logsumexp([np.log(1 - omega), np.log(omega) + cdf_val])
        else:
            target += np.log(omega) + st.norm.logpdf(i, mu, sigma)
                                              

    return target

def log_like_iid_normal(params, n, thresh):
    """Log likelihood for i.i.d. normal measurements.

    Parameters
    ----------
    params : array
        Parameters mu, sigma.
    n : array
        Array of data points.

    Returns
    -------
    output : float
        Log-likelihood.
    """
    mu, sigma = params

    if mu <= 0 or sigma <= 0:
        return -np.inf

    target = 0
    cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))
    
    for i in n:
        if i == thresh:
            target += cdf_val
        else:
            target += st.norm.logpdf(i, mu, sigma)

    return target
    

def log_like(params, n, model, thresh):
    """
    Model: 'lognormal', 'lognormal_mixed', 'normal', 'normal_mixed'
    """
    if model == 'lognormal':
        return log_like_iid_lognormal(params, n, thresh)
    elif model == 'lognormal_mixed':
        return log_like_iid_lognormal_mixed(params, n, thresh)
    elif model == 'normal':
        return log_like_iid_normal(params, n, thresh)
    elif model == 'normal_mixed':
        return log_like_iid_normal_mixed(params, n, thresh)
    else:
        raise ValueError('Pick an appropriate model!')
                                     

def mle_iid(n, model, thresh):
    """Perform maximum likelihood estimates for parameters for i.i.d.
    measurements of a chosen model;
    Models: 'lognormal', 'lognormal_mixed', 'normal', 'normal_mixed'
    """
    if model == 'lognormal':
        init_guess = np.array([2, 15])
    elif model == 'lognormal_mixed':
        init_guess = np.array([2, 15, 0.5])
    elif model == 'normal':
        init_guess = np.array([10, 15])
    elif model == 'normal_mixed':
        init_guess = np.array([10, 15, 0.5])
    else:
        raise ValueError('Pick an appropriate model!')

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        res = scipy.optimize.minimize(
            fun=lambda params, n: -log_like(params, n, model, thresh),
            x0=init_guess,
            args=(n,),
            method='Powell'
        )

        if res.success:
            return res.x
        else:
            raise RuntimeError('Convergence failed with message', res.message)


def gen_lognormal_mixed(params, thresh, size, rng):

    mu, sigma, omega = params
    num_max = rng.binomial(size, (1 - omega))
    y_max = np.ones(num_max) * thresh
    num_weib = size - num_max
    y_lognorm = st.lognorm.rvs(sigma, scale=np.exp(mu), size=num_weib)

    if len(y_max) == 0:
        y = y_lognorm
    else:
        y = np.concatenate((y_lognorm, y_max))
    y[y > thresh] = thresh
    return y


def gen_lognormal(params, thresh, size, rng):

    mu, sigma = params
    y = st.lognorm.rvs(sigma, scale=np.exp(mu), size=size)
    y[y > thresh] = thresh
    return y


def gen_normal(params, thresh, size, rng):

    mu, sigma = params
    y = st.norm.rvs(mu, sigma, size=size)
    y[y > thresh] = thresh
    return y
    

def draw_perm_sample(x, y):
    """Generate a permutation sample."""
    concat_data = np.concatenate((x, y))
    np.random.shuffle(concat_data)
    return concat_data[:len(x)], concat_data[len(x):]


def draw_perm_reps(x, y, stat_fun, size=1):
    """Generate array of permuation replicates."""
    return np.array([stat_fun(*draw_perm_sample(x, y)) for _ in range(size)])


def draw_perm_reps_diff_mean(x, y, size=1):
    """Generate array of permuation replicates."""
    out = np.empty(size)
    for i in range(size):
        x_perm, y_perm = draw_perm_sample(x, y)
        out[i] = np.abs(np.mean(x_perm) - np.mean(y_perm))
    return out

### Plotting characteristics

Specifying palettes and plotting order.

In [6]:
# Palette:
palette_CI = ["#000000", "#88B3B4", "#3A7E7F"]

# Plotting order:
order = {'WT_SPF': 1, 'ASO_SPF': 2, 'ASO_bC': 3}

## Pole descent

1. For ease of adapting the previous code, I am creating a dictionary with values for every group in a single test.

In [7]:
work_df = data_dict['Pole']
group_vals = {}

effect_1 = 'Genotype'
effect_2 = 'Microbiome'

effect1_lst = work_df[effect_1].unique()
effect2_lst = work_df[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = work_df.loc[(work_df[effect_1] == i) & (work_df[effect_2] == j), 'Measurement'].values
        if len(n) != 0:
            group_vals[name] = n

2. Calculating the AIC for a set of models: ['weibull', 'weibull_mixed', 'lognormal', 'lognormal_mixed', 'normal', 'normal_mixed', 'inverse_gamma', 'inverse_gamma_mixed']

In [8]:
models = ['lognormal', 'lognormal_mixed', 'normal', 'normal_mixed']

mles_dict = {}

AICs = pd.DataFrame(index = models)


for model in models:
    mles_dict[model] = {}
    for group in group_vals.keys():
        if len(group_vals[group]) != 0:
            params = mle_iid(group_vals[group], model=model, thresh=60.0)
        
            mles_dict[model][group] = params
    
            _llk = log_like(params, group_vals[group], model = model, thresh=60.0)
            AICs.loc[model, group] = -2 * (_llk - len(params))
    

AICs

/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:117: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:117: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:85: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:85: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))


,WT_SPF,ASO_SPF,ASO_SPF-bC
lognormal,61.851081,65.575906,69.005529
lognormal_mixed,63.851082,67.575907,71.005530
normal,58.811030,75.770853,71.050527
normal_mixed,60.811031,77.770854,73.050528


3. So far, I have decided to work with lognormal distribution, although the AIC favors inverse gamma. Lognormal gives a clearer image in the CI of parameters (however, this choice is a somewhat weak place in the analysis which I need to discuss with Justin).

### Lognormal

In [9]:
ln_mles = mles_dict['lognormal']
ln_mles

{'WT_SPF': array([1.75778048, 0.28697798]),
 'ASO_SPF': array([2.20105083, 0.43995417]),
 'ASO_SPF-bC': array([1.9587433 , 0.41583404])}

4. Calculating confidence intervals for the parameters of the chosen model.

Drawing bootstrap replicates of parameter MLEs for every group in the experiment.  
**Choose an appropriate amount for <code>n_jobs</code> according to the number of cores available on the machine. Windows users might need to set it to 1 to make it work. There is another option of using previously generated bootstrap samples - refer to the code below the following coding cell.**

In [10]:
bs_mle_samples_mx_ln = {}

for group in group_vals.keys():
    print(group)

    bs_mle_sample = bebi103.bootstrap.draw_bs_reps_mle(
        mle_iid,
        gen_lognormal,
        data=group_vals[group],
        mle_args=('lognormal', 60.0),
        gen_args=(60.0, ),
        size=10000,
        n_jobs=7
    )

    _df = pd.DataFrame(bs_mle_sample, columns=['mu', 'sigma'])
    _df.to_csv(os.path.join("..", "output", "benCom_pole_bs_mle_samples"+group+".csv"))

    bs_mle_samples_mx_ln[group] = bs_mle_sample

WT_SPF
ASO_SPF
ASO_SPF-bC


**If you are unable to run the code cell above or it takes too long, you can use the code below.**

In [11]:
for group in group_vals.keys():
    _df = pd.read_csv(os.path.join("..", "output", "benCom_pole_bs_mle_samples"+group+".csv"), index_col=0)
    bs_mle_samples_mx_ln[group] = _df.to_numpy()

5. Creating specific data structures to be able to plot the confidence intervals.

#### For plotting:

In [12]:
mu_lst_mx_ln = []
sigma_lst_mx_ln = []

mus_mx_ln = {}
sigmas_mx_ln = {}

for group in group_vals:
    _smpls = bs_mle_samples_mx_ln[group]
    _mu_CI = np.percentile(_smpls[:, 0], [2.5, 97.5])
    _sigma_CI = np.percentile(_smpls[:, 1], [2.5, 97.5])

    mu_mle, sigma_mle = ln_mles[group]

    _mu_dct = { 'label':group, 'conf_int':_mu_CI, 'estimate':mu_mle}
    _sigma_dct = { 'label':group, 'conf_int':_sigma_CI, 'estimate':sigma_mle}

    mu_lst_mx_ln.append(_mu_dct)
    sigma_lst_mx_ln.append(_sigma_dct)

### Graphical model assessment

In [13]:
bs_samples_mx_ln = {}
rng = np.random.default_rng()

for group in group_vals:

    params = ln_mles[group]
    single_samples = np.array([gen_lognormal(params, thresh=60.0, size=len(group_vals[group]), rng=rng) for _ in range(100000)])
    bs_samples_mx_ln[group] = single_samples


In [14]:
qqplots_mx_ln =[]
precdf_plots_mx_ln = []

for group in group_vals:

    p = bebi103.viz.qqplot(
        data=group_vals[group],
        samples=bs_samples_mx_ln[group],
        x_axis_label="time of descent",
        y_axis_label="time of descent",
        title=group + ' Q-Q plot'       
    )
    qqplots_mx_ln.append(p)

    p1 = bebi103.viz.predictive_ecdf(
        samples=bs_samples_mx_ln[group], 
        data=group_vals[group], 
        x_axis_label="time",
        title=group + ' predictive ECDF'
    )
    precdf_plots_mx_ln.append(p1)

qq_lt_mx_ln = bokeh.layouts.row(qqplots_mx_ln)

bokeh.io.show(qq_lt_mx_ln)

In [15]:
pe_lt_mx_ln = bokeh.layouts.row(precdf_plots_mx_ln)

bokeh.io.show(pe_lt_mx_ln)

### Generating 95% confidence intervals for the parameter MLEs

In [16]:
p = bebi103.viz.confints(
    mu_lst_mx_ln,
    title='mu 95%CI, lognormal model',
    palette=palette_CI
)

p.output_backend = "svg"

bokeh.io.show(p)

bokeh.io.export_svgs(p, filename=os.path.join("..", "figures", "benCom_pole_mu_CI.svg"))


bokeh.io.save(
    p,
    filename=('../figures/benCom_pole_mu_CI.html'),
    title='Bokeh plot',
    resources=bokeh.resources.CDN)

'/Users/anastasiaoguienko/git/amoiseyenko2025/figures/benCom_pole_mu_CI.html'

In [17]:
p = bebi103.viz.confints(
    sigma_lst_mx_ln,
    title='sigma 95%CI, lognormal model',
    palette=palette_CI
)
p.output_backend = "svg"

bokeh.io.show(p)
bokeh.io.export_svgs(p, filename=os.path.join("..", "figures", "benCom_pole_sigma_CI.svg"))


bokeh.io.save(
    p,
    filename=('../figures/benCom_pole_sigma_CI.html'),
    title='Bokeh plot',
    resources=bokeh.resources.CDN)

'/Users/anastasiaoguienko/git/amoiseyenko2025/figures/benCom_pole_sigma_CI.html'

## Beam cross

In [19]:
work_df = data_dict['Beam_time']
group_vals = {}

effect_1 = 'Genotype'
effect_2 = 'Microbiome'

effect1_lst = work_df[effect_1].unique()
effect2_lst = work_df[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = work_df.loc[(work_df[effect_1] == i) & (work_df[effect_2] == j), 'Measurement'].values
        if len(n) != 0:
            group_vals[name] = n

In [20]:
models = ['lognormal', 'lognormal_mixed', 'normal', 'normal_mixed']

mles_dict = {}

AICs = pd.DataFrame(index = models)


for model in models:
    mles_dict[model] = {}
    for group in group_vals.keys():
        if len(group_vals[group]) != 0:
            params = mle_iid(group_vals[group], model=model, thresh=60.0)
        
            mles_dict[model][group] = params
    
            _llk = log_like(params, group_vals[group], model = model, thresh=60.0)
            AICs.loc[model, group] = -2 * (_llk - len(params))
    

/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:117: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:117: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:117: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:85: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:85: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykerne

In [21]:
AICs

,WT_SPF,ASO_SPF,ASO_SPF-bC
lognormal,54.219822,58.760282,66.000916
lognormal_mixed,56.219823,60.760283,68.000917
normal,56.065822,55.945809,67.551649
normal_mixed,58.065823,57.945810,69.551650


### Lognormal

In [22]:
ln_mles = mles_dict['lognormal']
ln_mles

{'WT_SPF': array([1.32797484, 0.34200864]),
 'ASO_SPF': array([2.01263422, 0.38966455]),
 'ASO_SPF-bC': array([2.07108742, 0.33108659])}

Drawing bootstrap replicates of parameter MLEs for every group in the experiment.  
**Choose an appropriate amount for <code>n_jobs</code> according to the number of cores available on the machine. Windows users might need to set it to 1 to make it work. There is another option of using previously generated bootstrap samples - refer to the code below the following coding cell.**

In [23]:
bs_mle_samples_mx_ln = {}

for group in group_vals.keys():
    print(group)

    bs_mle_sample = bebi103.bootstrap.draw_bs_reps_mle(
        mle_iid,
        gen_lognormal,
        data=group_vals[group],
        mle_args=('lognormal', 60.0),
        gen_args=(60.0, ),
        size=10000,
        n_jobs=7
    )

    _df = pd.DataFrame(bs_mle_sample, columns=['mu', 'sigma'])
    _df.to_csv(os.path.join("..", "output", "benCom_beam_bs_mle_samples"+group+".csv"))

    bs_mle_samples_mx_ln[group] = bs_mle_sample

WT_SPF
ASO_SPF
ASO_SPF-bC


**If you are unable to run the code cell above or it takes too long, you can use the code below.**

In [24]:
for group in group_vals.keys():
    _df = pd.read_csv(os.path.join("..", "output", "benCom_beam_bs_mle_samples"+group+".csv"), index_col=0)
    bs_mle_samples_mx_ln[group] = _df.to_numpy()

#### For plotting:

In [25]:
mu_lst_mx_ln = []
sigma_lst_mx_ln = []

mus_mx_ln = {}
sigmas_mx_ln = {}

for group in group_vals:
    _smpls = bs_mle_samples_mx_ln[group]
    _mu_CI = np.percentile(_smpls[:, 0], [2.5, 97.5])
    _sigma_CI = np.percentile(_smpls[:, 1], [2.5, 97.5])

    mu_mle, sigma_mle = ln_mles[group]

    _mu_dct = { 'label':group, 'conf_int':_mu_CI, 'estimate':mu_mle}
    _sigma_dct = { 'label':group, 'conf_int':_sigma_CI, 'estimate':sigma_mle}

    mu_lst_mx_ln.append(_mu_dct)
    sigma_lst_mx_ln.append(_sigma_dct)

### Graphical model assessment

In [26]:
bs_samples_mx_ln = {}
rng = np.random.default_rng()

for group in group_vals:

    params = ln_mles[group]
    single_samples = np.array([gen_lognormal(params, thresh=60.0, size=len(group_vals[group]), rng=rng) for _ in range(100000)])
    bs_samples_mx_ln[group] = single_samples


In [27]:
qqplots_mx_ln =[]
precdf_plots_mx_ln = []

for group in group_vals:

    p = bebi103.viz.qqplot(
        data=group_vals[group],
        samples=bs_samples_mx_ln[group],
        x_axis_label="time to cross",
        y_axis_label="time to cross",
        title=group + ' Q-Q plot'       
    )
    qqplots_mx_ln.append(p)

    p1 = bebi103.viz.predictive_ecdf(
        samples=bs_samples_mx_ln[group], 
        data=group_vals[group], 
        x_axis_label="time",
        title=group + ' predictive ECDF'
    )
    precdf_plots_mx_ln.append(p1)

qq_lt_mx_ln = bokeh.layouts.row(qqplots_mx_ln)

bokeh.io.show(qq_lt_mx_ln)

In [28]:
pe_lt_mx_ln = bokeh.layouts.row(precdf_plots_mx_ln)

bokeh.io.show(pe_lt_mx_ln)

### Generating 95% confidence intervals for the parameter MLEs

In [29]:
p = bebi103.viz.confints(
    mu_lst_mx_ln,
    title='mu 95%CI, lognormal model',
    palette=palette_CI
)
p.output_backend = "svg"

bokeh.io.show(p)

bokeh.io.export_svgs(p, filename=os.path.join("..", "figures", "benCom_beam_mu_CI.svg"))


bokeh.io.save(
    p,
    filename=('../figures/benCom_beam_mu_CI.html'),
    title='Bokeh plot',
    resources=bokeh.resources.CDN)

'/Users/anastasiaoguienko/git/amoiseyenko2025/figures/benCom_beam_mu_CI.html'

In [30]:
p = bebi103.viz.confints(
    sigma_lst_mx_ln,
    title='sigma 95%CI, lognormal model',
    palette=palette_CI
)
p.output_backend = "svg"

bokeh.io.show(p)

bokeh.io.export_svgs(p, filename=os.path.join("..", "figures", "BenCom_beam_sigma_CI.svg"))


bokeh.io.save(
    p,
    filename=('../figures/BenCom_beam_sigma_CI.html'),
    title='Bokeh plot',
    resources=bokeh.resources.CDN)

'/Users/anastasiaoguienko/git/amoiseyenko2025/figures/BenCom_beam_sigma_CI.html'

## Adhesive removal

In [32]:
work_df = data_dict['Adhesive_removal']
group_vals = {}

effect_1 = 'Genotype'
effect_2 = 'Microbiome'

effect1_lst = work_df[effect_1].unique()
effect2_lst = work_df[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = work_df.loc[(work_df[effect_1] == i) & (work_df[effect_2] == j), 'Measurement'].values
        if len(n) != 0:
            group_vals[name] = n

In [33]:
models = ['lognormal', 'lognormal_mixed', 'normal', 'normal_mixed']

mles_dict = {}

AICs = pd.DataFrame(index = models)


for model in models:
    mles_dict[model] = {}
    for group in group_vals.keys():
        if len(group_vals[group]) != 0:
            params = mle_iid(group_vals[group], thresh=30.0, model=model)
        
            mles_dict[model][group] = params
    
            _llk = log_like(params, group_vals[group], thresh=30.0, model = model)
            AICs.loc[model, group] = -2 * (_llk - len(params))
    

/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:53: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.lognorm.cdf(thresh, sigma, scale=np.exp(mu)))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:117: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:117: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:117: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:85: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r

In [34]:
AICs

,WT_SPF,ASO_SPF,ASO_SPF-bC
lognormal,29.595706,42.049856,30.437110
lognormal_mixed,31.595707,44.049856,32.437111
normal,33.072062,49.533965,31.176563
normal_mixed,35.072063,51.533965,33.176564


### Lognormal

In [35]:
ln_mles = mles_dict['lognormal']
ln_mles

{'WT_SPF': array([0.45351541, 0.36086028]),
 'ASO_SPF': array([1.16274614, 0.42649883]),
 'ASO_SPF-bC': array([0.75093717, 0.31566815])}

Drawing bootstrap replicates of parameter MLEs for every group in the experiment.  
**Choose an appropriate amount for <code>n_jobs</code> according to the number of cores available on the machine. Windows users might need to set it to 1 to make it work. There is another option of using previously generated bootstrap samples - refer to the code below the following coding cell.**

In [36]:
bs_mle_samples_mx_ln = {}

for group in group_vals.keys():
    print(group)

    bs_mle_sample = bebi103.bootstrap.draw_bs_reps_mle(
        mle_iid,
        gen_lognormal,
        data=group_vals[group],
        mle_args=('lognormal', 30.0),
        gen_args=(30.0, ),
        size=10000,
        n_jobs=7
    )

    _df = pd.DataFrame(bs_mle_sample, columns=['mu', 'sigma'])
    _df.to_csv(os.path.join("..", "output", "benCom_adhesive_bs_mle_samples"+group+".csv"))

    bs_mle_samples_mx_ln[group] = bs_mle_sample

WT_SPF
ASO_SPF
ASO_SPF-bC


**If you are unable to run the code cell above or it takes too long, you can use the code below.**

In [37]:
for group in group_vals.keys():
    _df = pd.read_csv(os.path.join("..", "output", "benCom_adhesive_bs_mle_samples"+group+".csv"), index_col=0)
    bs_mle_samples_mx_ln[group] = _df.to_numpy()

#### For plotting:

In [38]:
mu_lst_mx_ln = []
sigma_lst_mx_ln = []

mus_mx_ln = {}
sigmas_mx_ln = {}

for group in group_vals:
    _smpls = bs_mle_samples_mx_ln[group]
    _mu_CI = np.percentile(_smpls[:, 0], [2.5, 97.5])
    _sigma_CI = np.percentile(_smpls[:, 1], [2.5, 97.5])

    mu_mle, sigma_mle = ln_mles[group]

    _mu_dct = { 'label':group, 'conf_int':_mu_CI, 'estimate':mu_mle}
    _sigma_dct = { 'label':group, 'conf_int':_sigma_CI, 'estimate':sigma_mle}

    mu_lst_mx_ln.append(_mu_dct)
    sigma_lst_mx_ln.append(_sigma_dct)

### Graphical model assessment

In [39]:
bs_samples_mx_ln = {}
rng = np.random.default_rng()

for group in group_vals:

    params = ln_mles[group]
    single_samples = np.array([gen_lognormal(params, thresh=30.0, size=len(group_vals[group]), rng=rng) for _ in range(100000)])
    bs_samples_mx_ln[group] = single_samples


In [40]:
qqplots_mx_ln =[]
precdf_plots_mx_ln = []

for group in group_vals:

    p = bebi103.viz.qqplot(
        data=group_vals[group],
        samples=bs_samples_mx_ln[group],
        x_axis_label="time of removal",
        y_axis_label="time of removal",
        title=group + ' Q-Q plot'       
    )
    qqplots_mx_ln.append(p)

    p1 = bebi103.viz.predictive_ecdf(
        samples=bs_samples_mx_ln[group], 
        data=group_vals[group], 
        x_axis_label="time",
        title=group + ' predictive ECDF'
    )
    precdf_plots_mx_ln.append(p1)

qq_lt_mx_ln = bokeh.layouts.row(qqplots_mx_ln)

bokeh.io.show(qq_lt_mx_ln)

In [41]:
pe_lt_mx_ln = bokeh.layouts.row(precdf_plots_mx_ln)

bokeh.io.show(pe_lt_mx_ln)

### Generating 95% confidence intervals for the parameter MLEs

In [42]:
p = bebi103.viz.confints(
    mu_lst_mx_ln,
    title='mu 95%CI, lognormal model',
    palette=palette_CI
)
p.output_backend = "svg"

bokeh.io.show(p)

bokeh.io.export_svgs(p, filename=os.path.join("..", "figures", "benCom_adhesive_mu_CI.svg"))


bokeh.io.save(
    p,
    filename=('../figures/benCom_adhesive_mu_CI.html'),
    title='Bokeh plot',
    resources=bokeh.resources.CDN)

'/Users/anastasiaoguienko/git/amoiseyenko2025/figures/benCom_adhesive_mu_CI.html'

In [43]:
p = bebi103.viz.confints(
    sigma_lst_mx_ln,
    title='sigma 95%CI, lognormal model',
    palette=palette_CI
)
p.output_backend = "svg"

bokeh.io.show(p)

bokeh.io.export_svgs(p, filename=os.path.join("..", "figures", "benCom_adhesive_sigma_CI.svg"))


bokeh.io.save(
    p,
    filename=('../figures/benCom_adhesive_sigma_CI.html'),
    title='Bokeh plot',
    resources=bokeh.resources.CDN)

'/Users/anastasiaoguienko/git/amoiseyenko2025/figures/benCom_adhesive_sigma_CI.html'

## Wirehang

In [44]:
work_df = data_dict['Wirehang'].copy()

group_vals = {}

effect_1 = 'Genotype'
effect_2 = 'Microbiome'

effect1_lst = work_df[effect_1].unique()
effect2_lst = work_df[effect_2].unique()

work_df.loc[work_df['Measurement'] == 0, 'Measurement'] = np.nan
work_df = work_df.dropna()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = work_df.loc[(work_df[effect_1] == i) & (work_df[effect_2] == j), 'Measurement'].values
        if len(n) != 0:
            group_vals[name] = n


In [45]:
models = ['lognormal', 'lognormal_mixed', 'normal', 'normal_mixed']

mles_dict = {}

AICs = pd.DataFrame(index = models)


for model in models:
    mles_dict[model] = {}
    for group in group_vals.keys():
        if len(group_vals[group]) != 0:
            params = mle_iid(group_vals[group], thresh=60.0, model=model)
        
            mles_dict[model][group] = params
    
            _llk = log_like(params, group_vals[group], thresh=60.0, model = model)
            AICs.loc[model, group] = -2 * (_llk - len(params))
    

/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:117: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_64575/1167199481.py:85: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(thresh, mu, sigma))


In [46]:
AICs

,WT_SPF,ASO_SPF,ASO_SPF-bC
lognormal,63.758671,64.689100,80.513261
lognormal_mixed,65.758678,66.689101,82.513262
normal,65.526137,63.707374,81.372421
normal_mixed,66.341618,65.707374,83.372422


### Lognormal

In [47]:
ln_mles = mles_dict['lognormal']
ln_mles

{'WT_SPF': array([4.66667456, 1.31092966]),
 'ASO_SPF': array([2.51087848, 0.30998804]),
 'ASO_SPF-bC': array([2.43866507, 0.51193835])}

Drawing bootstrap replicates of parameter MLEs for every group in the experiment.  
**Choose an appropriate amount for <code>n_jobs</code> according to the number of cores available on the machine. Windows users might need to set it to 1 to make it work. There is another option of using previously generated bootstrap samples - refer to the code below the following coding cell.**

In [48]:
bs_mle_samples_mx_ln = {}

for group in group_vals.keys():
    print(group)

    bs_mle_sample = bebi103.bootstrap.draw_bs_reps_mle(
        mle_iid,
        gen_lognormal,
        data=group_vals[group],
        mle_args=('lognormal', 60.0),
        gen_args=(60.0, ),
        size=10000,
        n_jobs=7
    )

    _df = pd.DataFrame(bs_mle_sample, columns=['mu', 'sigma'])
    _df.to_csv(os.path.join("..", "output", "benCom_wirehang_bs_mle_samples"+group+".csv"))

    bs_mle_samples_mx_ln[group] = bs_mle_sample

WT_SPF
ASO_SPF
ASO_SPF-bC


**If you are unable to run the code cell above or it takes too long, you can use the code below.**

In [49]:
for group in group_vals.keys():
    _df = pd.read_csv(os.path.join("..", "output", "benCom_wirehang_bs_mle_samples"+group+".csv"), index_col=0)
    bs_mle_samples_mx_ln[group] = _df.to_numpy()

#### For plotting:

In [50]:
mu_lst_mx_ln = []
sigma_lst_mx_ln = []

mus_mx_ln = {}
sigmas_mx_ln = {}

for group in group_vals:
    _smpls = bs_mle_samples_mx_ln[group]
    _mu_CI = np.percentile(_smpls[:, 0], [2.5, 97.5])
    _sigma_CI = np.percentile(_smpls[:, 1], [2.5, 97.5])

    mu_mle, sigma_mle = ln_mles[group]

    _mu_dct = { 'label':group, 'conf_int':_mu_CI, 'estimate':mu_mle}
    _sigma_dct = { 'label':group, 'conf_int':_sigma_CI, 'estimate':sigma_mle}

    mu_lst_mx_ln.append(_mu_dct)
    sigma_lst_mx_ln.append(_sigma_dct)

### Graphical model assessment

In [51]:
bs_samples_mx_ln = {}
rng = np.random.default_rng()

for group in group_vals:

    params = ln_mles[group]
    single_samples = np.array([gen_lognormal(params, thresh=60.0, size=len(group_vals[group]), rng=rng) for _ in range(100000)])
    bs_samples_mx_ln[group] = single_samples


In [52]:
qqplots_mx_ln =[]
precdf_plots_mx_ln = []

for group in group_vals:

    p = bebi103.viz.qqplot(
        data=group_vals[group],
        samples=bs_samples_mx_ln[group],
        x_axis_label="time of descent",
        y_axis_label="time of descent",
        title=group + ' Q-Q plot'       
    )
    qqplots_mx_ln.append(p)

    p1 = bebi103.viz.predictive_ecdf(
        samples=bs_samples_mx_ln[group], 
        data=group_vals[group], 
        x_axis_label="time",
        title=group + ' predictive ECDF'
    )
    precdf_plots_mx_ln.append(p1)

qq_lt_mx_ln = bokeh.layouts.row(qqplots_mx_ln)

bokeh.io.show(qq_lt_mx_ln)

In [53]:
pe_lt_mx_ln = bokeh.layouts.row(precdf_plots_mx_ln)

bokeh.io.show(pe_lt_mx_ln)

### Generating 95% confidence intervals for the parameter MLEs

In [54]:
p = bebi103.viz.confints(
    mu_lst_mx_ln,
    title='mu 95%CI, lognormal model',
    palette=palette_CI
)
p.output_backend = "svg"

bokeh.io.show(p)

bokeh.io.export_svgs(p, filename=os.path.join("..", "figures", "benCom_wirehang_mu_CI.svg"))


bokeh.io.save(
    p,
    filename=('../figures/benCom_wirehang_mu_CI.html'),
    title='Bokeh plot',
    resources=bokeh.resources.CDN)

'/Users/anastasiaoguienko/git/amoiseyenko2025/figures/benCom_wirehang_mu_CI.html'

In [55]:
p = bebi103.viz.confints(
    sigma_lst_mx_ln,
    title='sigma 95%CI, lognormal model',
    palette=palette_CI
)
p.output_backend = "svg"

bokeh.io.show(p)

bokeh.io.export_svgs(p, filename=os.path.join("..", "figures", "benCom_wirehang_sigma_CI.svg"))


bokeh.io.save(
    p,
    filename=('../figures/benCom_wirehang_sigma_CI.html'),
    title='Bokeh plot',
    resources=bokeh.resources.CDN)

'/Users/anastasiaoguienko/git/amoiseyenko2025/figures/benCom_wirehang_sigma_CI.html'